In [1]:
import os
import re
import json
import random
import sys
import asyncio
import pickle
import datetime
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel
from colorama import Fore, Style

c:\Users\HAOXUAN\miniconda3\envs\or\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [3]:
DATA_DIR = '../data'
DATASET_NAME = 'NL4OPT' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'nl4opt.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in demo_samples]
answers = [a for _, a in demo_samples]

2024-09-16 18:15:15.250 | DEBUG    | utils:read_txt_file:14 - Reading file: ../data\NL4OPT\nl4opt.txt
2024-09-16 18:15:15.251 | DEBUG    | utils:read_txt_file:16 - File read successfully: ../data\NL4OPT\nl4opt.txt
2024-09-16 18:15:15.252 | INFO     | utils:get_nl4opt_qas:35 - Number of questions: 245
2024-09-16 18:15:15.252 | INFO     | utils:get_nl4opt_qas:36 - Number of answers: 245
2024-09-16 18:15:15.253 | INFO     | utils:get_demo_and_test_samples:47 - Number of demo samples: 20
2024-09-16 18:15:15.253 | INFO     | utils:get_demo_and_test_samples:48 - Number of test samples: 225


### GPT-4o-mini + 2-shots CoT + Structured Output (on 20 samples, error < 1%) = 85%

### GPT-4o-mini + 2-shots CoT + Structured Output (on 20 samples, error < 1.5%) = 90%

In [6]:
# class VarRecStep(BaseModel):
#     reasoning: str

class VarRecg(BaseModel):
    # steps: list[VarRecStep]
    var_recg: str
    
# class ConObjStep(BaseModel):
#     reasoning: str

class ConObj(BaseModel):
    # steps: List[ConObjStep]
    evidence: List[str]
    math_expression: List[str]

# class CodeStep(BaseModel):
#     reasoning: str

class Code(BaseModel):
    # steps: List[CodeStep]
    reasoning: List[str]
    code: str     

class MultiStepInOnePrompt(BaseModel):
    var_recg: VarRecg
    con_obj: ConObj
    code: Code

In [7]:
sys_prompt = """You are an expert in optimization problems and domain specific language generation. Your task is to convert the textual optimization text into a piece of code.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x1 = solver.IntVar(0, solver.infinity(), 'x1')
x2 = solver.IntVar(0, solver.infinity(), 'x2')
solver.Add(46 * x1 + 13 * x2 <= 1345)
solver.Add(43 * x1 + 4 * x2 <= 346)
solver.Add(56 * x1 + 45 * x2 <= 1643)
objective = solver.Objective()
objective.SetCoefficient(x1, 10.0)
objective.SetCoefficient(x2, 15.0)
objective.SetMaximization()

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
CODE:
x1 = solver.IntVar(5, solver.infinity(), 'x1')
x2 = solver.IntVar(10, solver.infinity(), 'x2')
solver.Add(x1 + x2 <= 50)
solver.Add(x1 >= 5)
solver.Add(x2 >= 10)
solver.Add(x2 <= 2 * x1)
objective = solver.Objective()
objective.SetCoefficient(x1, 2.0)   
objective.SetCoefficient(x2, 4.0)
objective.SetMaximization()
"""

sys_prompt = sys_prompt + example + "\nPlease finish the task think step by step."
print(sys_prompt)

You are an expert in optimization problems and domain specific language generation. Your task is to convert the textual optimization text into a piece of code.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x1 = solver.IntVar(0, solver.infinity(), 'x1')
x2 = solver.IntVar(0, solver.infinity(), 'x2')
solver.Add(46 * x1 + 13 * x2 <= 1345)
solver.Add(43 * x1 + 4 * x2 <= 346)
solver.Ad

In [8]:
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": "QUESTION: " + questions[0]}
    ],
    temperature=0,
    response_format=Code,
)

In [11]:
result = response.choices[0].message.parsed
print(result)
# print(Fore.RED + f"Result: {questions[0]}")
# print(Fore.GREEN + f"VarRecg: {result.var_recg.var_recg}")
# print(Fore.CYAN + f"ConObj: {result.con_obj.math_expression}")
print(Fore.YELLOW + f"Code: {result.code}" + Style.RESET_ALL)

reasoning=['Define variables for the number of containers of each type of oil (Oil Max and Oil Max Pro).', 'Set up constraints based on the available amounts of substances A, B, and C.', 'Define the objective function to maximize profit based on the profit per container of each type of oil.'] code="x1 = solver.IntVar(0, solver.infinity(), 'x1')\nx2 = solver.IntVar(0, solver.infinity(), 'x2')\nsolver.Add(46 * x1 + 13 * x2 <= 1345)\nsolver.Add(43 * x1 + 4 * x2 <= 346)\nsolver.Add(56 * x1 + 45 * x2 <= 1643)\nobjective = solver.Objective()\nobjective.SetCoefficient(x1, 10.0)\nobjective.SetCoefficient(x2, 15.0)\nobjective.SetMaximization()"
Code: x1 = solver.IntVar(0, solver.infinity(), 'x1')
x2 = solver.IntVar(0, solver.infinity(), 'x2')
solver.Add(46 * x1 + 13 * x2 <= 1345)
solver.Add(43 * x1 + 4 * x2 <= 346)
solver.Add(56 * x1 + 45 * x2 <= 1643)
objective = solver.Objective()
objective.SetCoefficient(x1, 10.0)
objective.SetCoefficient(x2, 15.0)
objective.SetMaximization()


In [12]:
batch_size = 8
lp_reasoning_list = []
for idx in tqdm(range(0, len(questions), batch_size)):
    batch = questions[idx:idx+batch_size]
    
    tasks = [asyncclient.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0,
        response_format=Code,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": f"QUESTION: {q}"}
        ]) for q in batch
    ]

    combined_responses = await asyncio.gather(*tasks)
    lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

100%|██████████| 3/3 [00:12<00:00,  4.29s/it]


In [13]:
filename = 'multi_step_in_one_prompt_result_one_shot-' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(lp_reasoning_list, f)

In [14]:
codes = [lp_reasoning_list[i].code for i in range(len(lp_reasoning_list))]

In [15]:
prefix = """
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('GLOP') 
if not solver: raise
"""
                
suffix = """
status = solver.Solve()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [16]:
def clean_code(code: str) -> str:
    cleand_code = []
    for line in code.split('\n'):
        line = line.strip()
        if line.startswith('solver.Add') and not re.findall(r'<=|>=', line):
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        cleand_code.append(line)
    return '\n'.join(cleand_code)

In [17]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    solver = ex_locals["solver"]
    
    if ex_locals['status'] == ex_locals['pywraplp'].Solver.OPTIMAL:
        return solver.Objective().Value()
    else:
        return np.inf

In [18]:
pred_answers = []
for i, code_str in enumerate(codes):
    try:
        cleaned_code = clean_code(code_str)
        code = complement_code(cleaned_code)
        ans = execute_code(code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-16 18:18:52.885 | INFO     | __main__:<module>:7 - question 0 obtain answer
2024-09-16 18:18:52.886 | INFO     | __main__:<module>:7 - question 1 obtain answer
2024-09-16 18:18:52.887 | INFO     | __main__:<module>:7 - question 2 obtain answer
2024-09-16 18:18:52.888 | INFO     | __main__:<module>:7 - question 3 obtain answer
2024-09-16 18:18:52.888 | INFO     | __main__:<module>:7 - question 4 obtain answer
2024-09-16 18:18:52.889 | INFO     | __main__:<module>:7 - question 5 obtain answer
2024-09-16 18:18:52.890 | INFO     | __main__:<module>:7 - question 6 obtain answer
2024-09-16 18:18:52.890 | INFO     | __main__:<module>:7 - question 7 obtain answer
2024-09-16 18:18:52.891 | INFO     | __main__:<module>:7 - question 8 obtain answer
2024-09-16 18:18:52.891 | INFO     | __main__:<module>:7 - question 9 obtain answer
2024-09-16 18:18:52.892 | INFO     | __main__:<module>:7 - question 10 obtain answer
2024-09-16 18:18:52.893 | INFO     | __main__:<module>:7 - question 11 obta

In [25]:
print([round(pa, 4) for pa in pred_answers])

[547.6667, 166.6667, 950.0, 37083.3333, 342857.1429, 7000.0, 100.0, 12000.0, 480.0, 142.8571, 466.6667, inf, 66.6667, inf, 1500.0, 511.4286, 1072.0, 11250.0, 19.2308, 47.7778]


In [26]:
print(answers)

['540.0', '166.66666666666669', '950.0', '36900.0', '342750.0', '7000.0', '100.0', '11980.0', '480.0', '142.0', '465.0', inf, '67.0', inf, '1500.0', '511.42857142857133', '1060.0', '2500.0', '20.0', '-99999']


In [29]:
correct = []
for p, r in zip(pred_answers, answers):
    if p == 'Error':
        continue
    if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < 1.5e-2):
        correct.append(True)
    else:
        correct.append(False)

In [30]:
(sum(correct) / len(answers)) * 100

90.0